# Automatic Playlist Generation:
# A Content-Based Music Sequence Recommender System

## 1. Concept

#### A. Podcast-like Playlists:
- Categorical Tags (Genre, Era/Year, Label, Producers)
- Qualitative Tags (Dancebility, BPM, Key, Vocal/Instrumental)

#### B. Mixtape-like Playlists:  
- Audio Features
- Feature Similarity Measures   
    - Harmony
    - Rhythm
    - Sound
    - Instrumentation
    - Mood/Sentiment
    - Dynamic

#### C. DJ-Mixes:
- Start/Intro & End/Outro Features of Songs
- Beat Matching Features
- Song to Song Transition Features
- Story Telling Features over whole Song Sequence    
- Coherence Measures for Transitions & Sequences

## 2. Recommender Systems Overview: State of the Art Approaches

#### A. Two General Approaches:

- **Collaborative filtering:** Matrix Factorization, alternating least squares
- **Content-based approaches:** Input is music information (basis of songs and/or 
    existing playlists) fetched through Music Information Retrieval (MIR) processes


 B. What are the Recommendations / the generated Playlists based on?

- emotion / mood
- genre
- user taste
- user similarity
- popularity


 C. More recent Approaches / Deep Learning Approaches

- Sequence-aware music recommendation:
    - Next track recommendatons
    - Automatic playlist continuation (APC)

## 3. Possible Datasets, Models & Feature Selection

  A. Datasets:

[**Melon Music Dataset**](https://github.com/MTG/melon-music-dataset)  
[last.fm Dataset](https://zenodo.org/record/6090214)  
[MTG Barcelona Datasets & Software](https://www.upf.edu/web/mtg/software-datasets)  
[Kaggle: Spotify Tracks Dataset](https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset?datasetId=2570056&sortBy=voteCount)  
[Kaggle: Spotify Playlists Dataset](https://www.kaggle.com/datasets/andrewmvd/spotify-playlists?datasetId=1720572&sortBy=voteCount)

  B. Python Audio Analysis (MIR) Packages: 

[**Essentia (ML Application ready)**](https://essentia.upf.edu/)  
[Essentia citing papers](https://essentia.upf.edu/research_papers.html)  
[**Librosa (lightweigth analysis)**](https://librosa.org/doc/main/feature.html)


  C. Youtube Tutorials:

[Spotify Playlist Generation](https://www.youtube.com/watch?v=3vvvjdmBoyc&list=PL-wATfeyAMNrTEgZyfF66TwejA0MRX7x1&index=2)  
[Librosa Music Analysis](https://www.youtube.com/watch?v=MhOdbtPhbLU)

## 4. Content-Based Recommendation

**Reasoning:** *Cold-start problem for metadata-based recommendation systems using only user-generated metadata*  
**Solution:** *Find underlying features of audio/music by MIR*  
**High-level Features:** *genre, mood, instrument(s), vocals, gender of singing voice, lyrics, ...*  
**Low-level Features:** *MFCC, ZCR, Spectral Coefficients, mixability*



## 5. Strategy

    1. Obtain mixes data

    2. Get songs for each mix  

    3. Analyze song / sequence data for content-based Recommendation system  

    4. Produce playlists  

    5. Compare to baseline model  

    (6. Produce dj-mix with transitions)

map mixes songs to spotify

extract items featurers matrix for mixes

## CODE

In [36]:
import os
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pylab as plt
%matplotlib inline

import librosa
import librosa.display

from IPython.display import Audio
import ipywidgets as widgets

  Librosa: MIR Library

  The Chromagram

In [37]:
# y_harmonic, y_percussive = librosa.effects.hpss(y)

In [38]:
# C = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr)

In [39]:
# plt.figure(figsize=(16,4))
# librosa.display.specshow(C, sr=sr, x_axis='time', y_axis='chroma', vmin=0, vmax=1)
# plt.title('Chromagram for: {}'.format(song_name))
# plt.colorbar()
# plt.show()

#### mixesDb Extractions

In [40]:
from eda import Mixes, Tracks

In [41]:
mixes = Mixes('../data/djmix-dataset.json', min_files=5, max_files=20)

In [42]:
mixes

,mix_id,tracks_identified,tracks_total,tracklist,genre,year,available_files,files_count
0,mix0018,13,14,"[7ZOqI15csjw, None, n4xiIDK8SoI, 4o4joooH5tg, ...",Techno,1995,"[7ZOqI15csjw, n4xiIDK8SoI, 4o4joooH5tg, VKLZ4g...",13
1,mix0066,10,13,"[None, qycAC_6Bbto, _QFaJ10-IfA, mSCtgcTBJpw, ...",Progressive,2000,"[qycAC_6Bbto, _QFaJ10-IfA, mSCtgcTBJpw, Y_Gx61...",9
2,mix0126,14,14,"[Nz2SHyePD2o, tcwjlNMPxMM, RV8jSPQtOsU, AdM6Kn...",Progressive,2002,"[Nz2SHyePD2o, tcwjlNMPxMM, RV8jSPQtOsU, AdM6Kn...",14
3,mix0136,19,20,"[NZWw0wD3Bwk, oBXfAPtGJkg, gINgF2YbAzo, zxxx-U...",Trance,2002,"[NZWw0wD3Bwk, vq4kUjqBh10, tBnf98KdpWM, tcwjlN...",5
4,mix0141,17,18,"[J-IMugmfpU0, 5jxRNG4WqgM, jzZsomI_tUU, G5RCTu...",Progressive Trance,2003,"[J-IMugmfpU0, 5jxRNG4WqgM, jzZsomI_tUU, G5RCTu...",17
...,...,...,...,...,...,...,...,...
273,mix4449,10,13,"[FwvHof6QbVM, 0U5yzv3o6ug, 0gEC_jJ9a9g, None, ...",Various,2019,"[FwvHof6QbVM, 0U5yzv3o6ug, 0gEC_jJ9a9g, 8dgEKH...",8
274,mix4646,10,11,"[EPC1u4vQ6Kk, None, InE1GUFz_fA, HrYWeIdEyVE, ...",Various,2019,"[EPC1u4vQ6Kk, InE1GUFz_fA, HrYWeIdEyVE, zvFVOv...",10
275,mix4699,10,12,"[VDoIfDghpyk, kU65WDeoMps, hOulfSfBhPc, vMZz7X...",Jungle,2019,"[VDoIfDghpyk, kU65WDeoMps, hOulfSfBhPc, vMZz7X...",10
276,mix4702,15,19,"[glAEPjCTr_I, HfdpXQ6cSyw, bJSAqj_aEes, None, ...",Tech House,2019,"[glAEPjCTr_I, HfdpXQ6cSyw, 0jfovV5K0fk, TLsxXP...",10


In [43]:
mixes.groupby('genre').mix_id.count()

genre
Acid                   2
Chill Out              1
Deep House            25
Deep Tech House        7
Detroit Techno         1
Disco                  5
Drum & Bass           15
Dub                    1
Dubstep                2
Electro                3
Ghetto                 1
Hard Techno            1
House                 49
House (Old School)     3
IDM                    2
Jungle                 1
Minimal                4
Pop                    1
Progressive            6
Progressive House     26
Progressive Trance     4
Psytrance              1
Rare Groove            1
Soul                   1
Tech House            59
Techno                45
Trance                 5
Various                6
Name: mix_id, dtype: int64

## QUESTIONS:
        can I use this data to categorize other songs?
        This genre-list is pretty long, maybe it might be 
        usefull for genre classification beforehand.
#### MAKE TRACKS DB WITH GENRES
        label all tracks with mix_ids, genres, what else?
#### USER-ITEM-LIKE MATRIX FOR MIXES AND TRACKS

## Librosa Library: Features

**Spectral Features:**

1. `chroma_stft`: Computes the chromagram (pitch class profile) from a waveform or power spectrogram. Useful for music genre classification and chord recognition.

2. `chroma_cqt`: Computes the constant-Q chromagram, which is a chromagram variant based on the constant-Q transform. It is useful for analyzing musical audio.

3. `chroma_cens`: Computes the chroma variant called "Chroma Energy Normalized" (CENS). It provides a more robust representation of tonal content and is useful for tasks like cover song identification.

4. `chroma_vqt`: Computes the variable-Q chromagram, which is a chromagram variant based on the variable-Q transform. It is useful for analyzing musical audio.

5. `melspectrogram`: Computes a mel-scaled spectrogram, which represents the power spectrum of a signal in mel-frequency bins. It is commonly used in speech and music analysis.

6. `mfcc`: Computes Mel-frequency cepstral coefficients (MFCCs), which are a compact representation of the spectral envelope of a signal. They are widely used in speech and audio processing tasks such as speech recognition and speaker identification.

7. `rms`: Computes the root-mean-square (RMS) value for each frame in an audio signal. It provides a measure of the overall energy level of the signal.

8. `spectral_centroid`: Computes the spectral centroid, which represents the center of mass of the spectrum. It provides information about the "brightness" of a sound and is useful for tasks like instrument recognition.

9. `spectral_bandwidth`: Computes the spectral bandwidth, which measures the spread of the spectrum around the spectral centroid. It provides information about the "width" of a sound and is useful for tasks like timbre characterization.

10. `spectral_contrast`: Computes spectral contrast, which measures the difference in amplitude between peaks and valleys in the spectrum. It is useful for tasks like music genre classification and instrument recognition.

11. `spectral_flatness`: Computes the spectral flatness, which quantifies the "tonality" of a sound by measuring the ratio of the geometric mean to the arithmetic mean of the spectrum.

12. `spectral_rolloff`: Computes the roll-off frequency, which is the frequency below which a specified percentage of the total spectral energy lies. It provides information about the shape and brightness of the spectrum.

13. `poly_features`: Computes the coefficients of fitting an nth-order polynomial to the columns of a spectrogram. It can be used for tasks like pitch estimation and separation of harmonic and percussive components.

14. `tonnetz`: Computes the tonal centroid features (tonnetz), which represent the harmonic relationships between musical notes. It is useful for tasks like chord recognition and key estimation.

15. `zero_crossing_rate`: Computes the zero-crossing rate, which measures the rate at which a signal changes sign. It is often used as a simple feature for tasks like speech and music onset detection.


**Rhythm Features:**

16. `tempo`: Estimates the tempo (beats per minute) of an audio signal. It is useful for tasks like music classification and tempo-based audio processing.

17. `tempogram`: Computes the tempogram, which is a local autocorrelation of the onset strength envelope. It provides a visualization of rhythmic patterns in the audio signal.

18. `fourier_tempogram`: Computes the Fourier tempogram, which is the short-time Fourier transform of the onset strength envelope. It provides a spectrogram-like representation of rhythmic patterns in the audio signal.

19. `tempogram_ratio`: Computes temp

## Brain Storming
    Song Feature Dataframe
    Mix Feature Matrix
    Model Selection
    INPUT OUTPUT: 2-3 Songs -> recoomend existing mix
    HAT ARE FEATURES THAT MAKE A SONG FIT INTO A PLAYLIST? -> TRACK SILMILIARITY FEATURES
    WHAT ARE FEATURES THAT ORDER A PLAYLIST TO A SEQUENCE? -> MIX SEQUENCE FEATURES, TRANSITION MATRIX?

## REVERSE ENGINEERING OF MIX FEATURES: TIMESERIES TRENDS
    HOW DO THE FEATURES EVOLVE OVER A SET?
    HOW DO I WANT THE FEATURES TO EVOLVE OVER A SET?

#### BASELINE MODEL: BPM FILTER & RANDOM SAMPLING

In [44]:
init_file = '../audio/Ezra - Garten meiner Fantasie.wav'

In [45]:
init_file_name = init_file.split('/')[-1]

In [46]:
init_file_name

'Ezra - Garten meiner Fantasie.wav'

In [47]:
y, sr = librosa.load(init_file)

# Estimate BPM
init_tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

print("Estimated Starting BPM:", init_tempo)

Estimated Starting BPM: 107.666015625


In [48]:
min_tempo = init_tempo * 0.9
min_tempo

96.8994140625

In [49]:
max_tempo = init_tempo * 1.0825
max_tempo

116.5484619140625

In [50]:
files_folder = '../audio/'

In [51]:
import os
import librosa
import pandas as pd

In [52]:
synchable_files = []
for file in os.listdir(files_folder):
    try:
        y, sr = librosa.load(''.join([files_folder,file]))
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        synchable_files.append([file, tempo])
    except:
        continue

/var/folders/sn/zz32wyd938j2bblqhcs5jn180000gn/T/ipykernel_5682/3243987995.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(''.join([files_folder,file]))
/Users/bjornbraun/anaconda3/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/var/folders/sn/zz32wyd938j2bblqhcs5jn180000gn/T/ipykernel_5682/3243987995.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(''.join([files_folder,file]))
/Users/bjornbraun/anaconda3/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
Note: Illegal Audio-MPEG-Header 0xddb88486

In [53]:
possible_files = pd.DataFrame(synchable_files)

In [54]:
possible_files.columns=['file', 'bpm']

In [55]:
files_sorted = possible_files.sort_values(by='bpm')

In [56]:
files_sorted

,file,bpm
46,Black Milk - Detroit's New Dance Show.mp3,69.837416
51,Neroche - Take This Book And Burn It.wav,73.828125
40,Stereo Total - Wir tanzen im 4-Eck.mp3,78.302557
7,Stereo Total - Musique automatique.mp3,78.302557
49,DAF - Der Mussolini.mp3,78.302557
...,...,...
57,The Velvet Underground- Venus in Furs.flac,143.554688
53,AXEL F (Knight Horse Remix).mp3,151.999081
52,DAENGER - Kapitalismus stinkt (DAENGER Repair)...,172.265625
70,Saweetie - Best Friend (feat. Doja Cat).mp3,184.570312


In [57]:
filtered_files = files_sorted[(files_sorted['bpm'] >= min_tempo) & (files_sorted['bpm'] <= max_tempo)]

In [58]:
filtered_files

,file,bpm
65,Vince Staples - Norf Norf.m4a,99.384014
34,Tom Waits - Tango Till They're Sore.mp3,99.384014
71,Twerking Class Heroes - Baile en patines.wav,99.384014
61,Donovan - Season Of The Witch.mp3,99.384014
13,Madonna - La Isla Bonita.mp3,99.384014
10,Stereo Total - Liebe zu dritt.mp3,99.384014
54,Black Milk - Could It Be.mp3,99.384014
31,James Brown - The Boss.mp3,99.384014
0,Shygirl - UCKERS.mp3,99.384014
39,SAULT - Up All Night.mp3,103.359375


In [59]:
filtered_sample = filtered_files.query("file != @init_file_name").sample(n=12).sample(frac=1)

In [60]:
filtered_sample = filtered_sample.reset_index(drop=True)

In [61]:
filtered_sample

,file,bpm
0,Djedjotronic - Are Friends Electric feat Lokie...,112.347147
1,Fértil Discos - Barda - Las Cosas Que Hay Det...,103.359375
2,Vince Staples - Norf Norf.m4a,99.384014
3,Nelson Bell - Ruhig Bleiben.wav,112.347147
4,Shygirl - UCKERS.mp3,99.384014
5,Tom Waits - Tango Till They're Sore.mp3,99.384014
6,Twerking Class Heroes - Baile en patines.wav,99.384014
7,James Brown - The Boss.mp3,99.384014
8,Suuns - Up Past The Nursery.wav,103.359375
9,Bee Gees - Night Fever.flac,107.666016


In [62]:
init_row = pd.DataFrame({'file': [init_file_name], 'bpm': [init_tempo]})
filtered_sample = pd.concat([init_row, filtered_sample], ignore_index=True)

In [63]:
filtered_sample

,file,bpm
0,Ezra - Garten meiner Fantasie.wav,107.666016
1,Djedjotronic - Are Friends Electric feat Lokie...,112.347147
2,Fértil Discos - Barda - Las Cosas Que Hay Det...,103.359375
3,Vince Staples - Norf Norf.m4a,99.384014
4,Nelson Bell - Ruhig Bleiben.wav,112.347147
5,Shygirl - UCKERS.mp3,99.384014
6,Tom Waits - Tango Till They're Sore.mp3,99.384014
7,Twerking Class Heroes - Baile en patines.wav,99.384014
8,James Brown - The Boss.mp3,99.384014
9,Suuns - Up Past The Nursery.wav,103.359375


In [64]:
target_bpm = filtered_sample.bpm.mean()

In [65]:
filtered_sample.bpm.median()

103.359375

In [66]:
filtered_sample.bpm.std()

5.318925466166762

In [67]:
import pyrubberband as pyrb
import soundfile as sf


In [68]:
def adjust_tempo(input_file, output_file, target_bpm):
    y, sr = librosa.load(input_file)
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    current_bpm = librosa.beat.tempo(y=y, sr=sr, hop_length=512)[0]
    time_stretch_ratio = target_bpm / current_bpm
    y_stretched = librosa.effects.time_stretch(y=y, rate=time_stretch_ratio)
    sf.write(output_file, y_stretched, sr)


In [69]:
for file in filtered_sample['file']:
    try:
        input_file = ''.join([files_folder, file])
        output_file = ''.join([files_folder, 'bpm_adjusted_', file])
        adjust_tempo(input_file, output_file, target_bpm)
    except:
        continue

/var/folders/sn/zz32wyd938j2bblqhcs5jn180000gn/T/ipykernel_5682/1773932158.py:4: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  current_bpm = librosa.beat.tempo(y=y, sr=sr, hop_length=512)[0]
/var/folders/sn/zz32wyd938j2bblqhcs5jn180000gn/T/ipykernel_5682/1773932158.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(input_file)
/Users/bjornbraun/anaconda3/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [70]:
bpm_adjusted_files = []
for file in os.listdir(files_folder):
    try:
        if file.startswith('bpm_adjusted'):
            y, sr = librosa.load(''.join([files_folder,file]))
            tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
            bpm_adjusted_files.append([file, tempo])
    except:
        continue

In [71]:
bpm_adjusted_files

[['bpm_adjusted_Suuns - Up Past The Nursery.wav', 107.666015625],
 ['bpm_adjusted_Nelson Bell - Ruhig Bleiben.wav', 103.359375],
 ['bpm_adjusted_Marie Davidson - The Psychologist.mp3', 103.359375],
 ["bpm_adjusted_Tom Waits - Tango Till They're Sore.mp3", 103.359375],
 ['bpm_adjusted_Fértil Discos - Barda - Las Cosas Que Hay Detras del Sol (feat Nick Drake).wav',
  103.359375],
 ['bpm_adjusted_Djedjotronic - Are Friends Electric feat Lokier.flac',
  103.359375],
 ['bpm_adjusted_Dan Bay - Wladimir Trump.wav', 107.666015625],
 ['bpm_adjusted_Ezra - Garten meiner Fantasie.wav', 103.359375],
 ['bpm_adjusted_Bee Gees - Night Fever.flac', 107.666015625],
 ['bpm_adjusted_Twerking Class Heroes - Baile en patines.wav', 107.666015625],
 ['bpm_adjusted_Shygirl - UCKERS.mp3', 103.359375],
 ['bpm_adjusted_James Brown - The Boss.mp3', 103.359375]]

In [1]:
import madmom